# Challenge

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

1. Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
2. What metric would you report for this dataset?
3. What is its value?

In [19]:
# Import libraries
import pandas as pd
import hvplot as hv
import plotly.express as px
from pathlib import Path

In [2]:
# Read in csv with 'created_at' column as index
file_path = Path('Resources/Shopify-Data-Science-Challenge.csv')
df = pd.read_csv(file_path, index_col='created_at', infer_datetime_format=True, parse_dates=True)
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method
created_at,,,,,,
2017-03-13 12:36:56,1,53,746,224,2,cash
2017-03-03 17:38:52,2,92,925,90,1,cash
2017-03-14 04:23:56,3,44,861,144,1,cash
2017-03-26 12:43:37,4,18,935,156,1,credit_card
2017-03-01 04:35:11,5,18,883,156,1,credit_card


In [3]:
# Reset index and drop times from datetime 'created_at' column
df.reset_index(inplace=True)
df['created_at'] = df['created_at'].dt.date

In [4]:
# Set 'created_at' column back to index
df.set_index('created_at', inplace=True)

In [5]:
# Check DataFrame
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method
created_at,,,,,,
2017-03-13,1,53,746,224,2,cash
2017-03-03,2,92,925,90,1,cash
2017-03-14,3,44,861,144,1,cash
2017-03-26,4,18,935,156,1,credit_card
2017-03-01,5,18,883,156,1,credit_card


In [6]:
# Check nulls
df.isnull().sum()

order_id          0
shop_id           0
user_id           0
order_amount      0
total_items       0
payment_method    0
dtype: int64

In [7]:
# Check Average Order Value (AOV)
df['order_amount'].mean()

3145.128

In [15]:
# Check max date
df.index.max()

datetime.date(2017, 3, 30)

In [9]:
# Check min date
df.index.min()

datetime.date(2017, 3, 1)

In [10]:
#Check number of transactions
df.tail()

,order_id,shop_id,user_id,order_amount,total_items,payment_method
created_at,,,,,,
2017-03-30,4996,73,993,330,2,debit
2017-03-16,4997,48,789,234,2,cash
2017-03-19,4998,56,867,351,3,cash
2017-03-16,4999,60,825,354,2,credit_card
2017-03-18,5000,44,734,288,2,debit


In [16]:
# Check total_items summary
df['total_items'].describe()

count    5000.00000
mean        8.78720
std       116.32032
min         1.00000
25%         1.00000
50%         2.00000
75%         3.00000
max      2000.00000
Name: total_items, dtype: float64

In [26]:
df['order_amount'].describe()

count      5000.000000
mean       3145.128000
std       41282.539349
min          90.000000
25%         163.000000
50%         284.000000
75%         390.000000
max      704000.000000
Name: order_amount, dtype: float64

In [31]:
items_outliers = px.box(
    df,
    x=df.index,
    y='total_items',
    title='Total Items Outliers Analysis',
    labels={'created_at': 'Date', 'total_items': 'Total Items'},
    color=df.index,
    boxmode='overlay'
    )

items_outliers

In [30]:
amount_outliers = px.box(
    df,
    x=df.index,
    y='order_amount',
    title='Order Amount Outliers Analysis',
    labels={'created_at': 'Date', 'order_amount': 'Order Amount'},
    color=df.index,
    boxmode='overlay'
    )

amount_outliers

In [25]:
def remove_outliers(data, columns, n_std):
    for col in columns:
        print('Working on column: {}'.format(col))

        mean = data[col].mean()
        sd = data[col].std()

        data=data[(data[col] <= mean+(n_std*sd))]

        return data